In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal, ogr, osr
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon,box
from shapely.geometry import mapping, shape
import cv2
import json
import geopandas as gpd
import numpy as np               # numerical array manipulation
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
import PIL.ImageDraw
import shutil
from itertools import product

import sys
from core.UNet import UNet
from core.losses import accuracy, dice_loss, IoU, recall, precision,F1_score
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.visualize import display_images

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis_bands
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis_bands.Configuration()

In [ ]:
# Load a pretrained model 
OPTIMIZER = adaDelta
OPTIMIZER=mixed_precision.LossScaleOptimizer(OPTIMIZER)
model = load_model(config.trained_model_path, custom_objects={'dice loss': dice_loss, 'accuracy':accuracy ,'recall':recall, 'F1_score':F1_score,'precision':precision,'IoU': IoU}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[dice_loss, accuracy,recall, precision, IoU])

In [ ]:
# Methods to add results of a patch   to    the total results of a larger area. 
#The operator could be min (useful if there are too many false positives), max (useful for tackle false negatives)
#res:mask [rows,cols] predition=np.squeeze(prediction[i], axis = -1) (col, row, wi, he) = batch_pos[i]
def addTOResult(res, prediction, row, col, he, wi,operator,ie_width):
    currValue = res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width]
    newPredictions = prediction[ie_width:he-ie_width, ie_width:wi-ie_width]
    
# IMPORTANT: MIN can't be used as long as the mask is initialed with 0!!!!! 
#If you want to use MIN initial the mask with -1 and handle the case of default value(-1) separately.
    if operator == 'min': # Takes the min of current prediction and new prediction for each pixel  
        currValue [currValue == -1] = 1 #Replace -1 with 1 in case of MIN  
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.minimum(currValue, newPredictions)
    elif operator == 'max':
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.maximum(currValue, newPredictions)
    else:
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = newPredictions  
    return (res)

In [ ]:
# Methods that actually makes the predictions
def predict_using_model(model, batch, batch_pos, mask,operator,ie_width):
#     print(batch)
    tm = np.stack(batch, axis = 0)
#     print(tm)
    prediction = model.predict(tm)
    for i in range(len(batch_pos)): 
        (col, row, wi, he) = batch_pos[i]
        p = np.squeeze(prediction[i], axis = -1)
        # Instead of replacing the current values with new values, use the user specified operator (MIN,MAX,REPLACE)
        mask = addTOResult(mask, p, row, col, he, wi,operator,ie_width)  
    return mask

def detect_lake(fullPath, NDWI_img, bands_img, operator ,width=512, height=512, stride = 256,ie_width=0,bandNum=1):
    nols, nrows = NDWI_img.meta['width'], NDWI_img.meta['height']
    meta = NDWI_img.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current NDWI_img',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, NDWI_img.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, bandNum),-1.0)#Add -1 padding in case of corner images
        read_ndwi_img=NDWI_img.read(window=window)/100
        read_bands_img = bands_img.read(window=window)/1000
#         hbh:change band sequence with the same as training data
        read_bands_img[[0,2],:]=read_bands_img[[2,0],:]
        temp_im = np.concatenate((read_ndwi_img,read_bands_img), axis=0)
        temp_im =  np.transpose(temp_im, axes=(1,2,0))

        patch[:window.height, :window.width] = temp_im   
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
        batch = []
        batch_pos = []

    return(mask, meta)

In [ ]:
def writeMaskToDisk(detected_mask, detected_meta, wp, write_as_type = 'uint8', th = 0.5, create_countors = False):
    # Convert to correct required before writing
    if 'float' in str(detected_meta['dtype']) and 'int' in write_as_type:
        print(f'Converting prediction from {detected_meta["dtype"]} to {write_as_type}, using threshold of {th}')#float32 to uint8
#         initial code have problem of big lake
        detected_mask[detected_mask<th]=0
        detected_mask[detected_mask>=th]=1
        detected_mask = detected_mask.astype(write_as_type)#'uint8'
        detected_meta['dtype'] =  write_as_type
    
    # compress tif
    detected_meta.update({"compress": 'lzw'})
    
    with rasterio.open(wp, 'w', **detected_meta) as outds:
        outds.write(detected_mask, 1)

In [ ]:
input_image_dir=r'F:/ndwi_int8_7000'
# input_image_dir=r'/media/nkd/data18931/ndwi_int8_13001_18931'
operator='max'
suffix='_iew{}'.format(config.ignore_edge_width)
for start in np.arange(1,7000,1000):
    
# start=11001
    end=start+1000
    for index in range(start,end):
        out_dir=os.path.join(config.output_dir,f'{start}_{end-1}')
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        filename=f'{config.ndwi_fn_st}_{index}{config.input_image_type}'
        fullPath=os.path.join(input_image_dir,filename)
        outputFile = os.path.join(out_dir, f'{config.output_prefix}{index}{suffix}{config.input_image_type}')
        if not os.path.isfile(outputFile) or config.overwrite_analysed_files: 
            with rasterio.open(fullPath) as NDWI:
                with rasterio.open(fullPath.replace(config.ndwi_fn_st,config.bands_fn_st)) as bands:
                    print(fullPath)
                    detectedMask, detectedMeta = detect_lake(fullPath,NDWI,bands,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE,ie_width=config.ignore_edge_width,bandNum=config.band_num)
                    writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
            print('File already analysed!', fullPath)
        
print('finish')